### Imports

In [1]:
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 166 µs (started: 2023-06-05 18:46:00 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = "greedy"
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

Exp:		 exp2
Method:		 greedy
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 2.37 ms (started: 2023-06-05 18:46:00 -05:00)


### Get common subgraphs

In [3]:
# read edges
dict_graphs = {}

for group in tqdm(groups_id):
    graphs = []
    for subgroup in tqdm(subgroups_id[group]):
        df_edges = pd.read_csv("output/{}/preprocessing/edges/edges_{}_{}.csv".format(exp, group, subgroup))
        G = nx.from_pandas_edgelist(df_edges, "source", "target", edge_attr="weight")
        # graph_detail(G)
        graphs.append(G)
    
    dict_graphs[group] = graphs

100%|██████████| 3/3 [01:17<00:00, 25.85s/it]

time: 1min 17s (started: 2023-06-05 18:46:00 -05:00)


In [4]:
# get subgraphs

dict_df_edges_filter = get_subgraphs_global(dict_graphs, groups_id)

100%|██████████| 4/4 [00:35<00:00,  8.88s/it]


Num. nodes: 4200
Num. edges: 135075



100%|██████████| 2/2 [00:46<00:00, 23.23s/it]


Num. nodes: 5579
Num. edges: 1100817



100%|██████████| 1/1 [00:12<00:00, 12.71s/it]


Num. nodes: 5710
Num. edges: 842738



100%|██████████| 3/3 [02:47<00:00, 55.80s/it]

time: 2min 47s (started: 2023-06-05 18:47:18 -05:00)


In [5]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,243.0404,267.1048
1,243.0404,401.2187
2,243.0404,661.3471
3,243.0404,267.0965
4,243.0404,416.1414


time: 15.4 ms (started: 2023-06-05 18:50:05 -05:00)


In [6]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [00:55<00:00, 18.60s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
133290,59.0137,59.0291,0.954735,0.928018,0.841093,0.601605,0.970185
131497,59.0137,349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
110528,59.0137,350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
132969,59.0137,350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
129629,59.0137,389.2739,0.823682,0.886260,0.723124,0.515230,0.917952


time: 56 s (started: 2023-06-05 18:50:05 -05:00)


### Filter by STD and average weight

In [7]:
# dispersion (std)

dict_df_common_edges_std = std_global_(dict_df_edges_filter_weight, exp, method, dimension, groups_id, th=0.3, plot=True, save=True)
dict_df_common_edges_std[groups_id[0]].head()

100%|██████████| 3/3 [01:04<00:00, 21.53s/it]


,source,target,weight
0,59.0137,59.0291,0.859127
1,59.0137,349.2758,0.701044
2,59.0137,350.2793,0.706849
3,59.0137,350.3033,0.736142
4,59.0137,389.2739,0.773249


time: 1min 4s (started: 2023-06-05 18:51:01 -05:00)


---

### STD-ANOVA

STD

In [8]:
# STD
    
""" df_edges_std = get_edges_std(SG, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter """

' df_edges_std = get_edges_std(SG, dir, group, subgroups, ddof=0)\ndf_edges_std\n\n# Filter by std (std < 0.3)\ndf_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]\ndf_edges_std_filter '

time: 3.64 ms (started: 2023-06-05 18:52:06 -05:00)


In [9]:
""" x = df_edges_std["std"]
plt.hist(x, bins=100)
plt.axvline(x=0.3, color="red", lw=1)
l = len(df_edges_std) - len(df_edges_std_filter)
t = len(df_edges_std)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_edges_std["std"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.3, color="red", lw=1)\nl = len(df_edges_std) - len(df_edges_std_filter)\nt = len(df_edges_std)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 3.4 ms (started: 2023-06-05 18:52:06 -05:00)


In [10]:
""" G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std.gexf".format(dir, group[0])) """

' G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])\ngraph_detail(G)\n\nlist_graphs.append(G.copy())\nnx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std.gexf".format(dir, group[0])) '

time: 2.98 ms (started: 2023-06-05 18:52:06 -05:00)


In [11]:
""" df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(G)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 3.35 ms (started: 2023-06-05 18:52:06 -05:00)


ANOVA

In [12]:
# ANOVA
""" df_nodes_anova = get_nodes_anova(G, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)
df_nodes_anova_filter """

' df_nodes_anova = get_nodes_anova(G, dir, group)\ndf_nodes_anova\n\n# Filter by ANOVA (p > 0.001)\ndf_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]\ndf_nodes_anova_filter\nnodes = list(df_nodes_anova_filter.index)\ndf_nodes_anova_filter '

time: 3.3 ms (started: 2023-06-05 18:52:06 -05:00)


In [13]:
""" x = df_nodes_anova["p-value"]
plt.hist(x, bins=100)
plt.axvline(x=0.001, color="red", lw=1)
l = len(df_nodes_anova) - len(df_nodes_anova_filter)
t = len(df_nodes_anova)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_nodes_anova["p-value"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.001, color="red", lw=1)\nl = len(df_nodes_anova) - len(df_nodes_anova_filter)\nt = len(df_nodes_anova)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 3.28 ms (started: 2023-06-05 18:52:06 -05:00)


In [14]:
""" H = G.subgraph(nodes)
graph_detail(H)
print(len(list(n for n in H.nodes() if H.degree(n) == 0)))

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std-anova.gexf".format(dir, group[0]))

df_edges_filter = nx.to_pandas_edgelist(H)

df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std-anova.csv".format(dir, group[0]), index=False)
df_edges_filter """

' H = G.subgraph(nodes)\ngraph_detail(H)\nprint(len(list(n for n in H.nodes() if H.degree(n) == 0)))\n\nlist_graphs.append(H.copy())\nnx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-std-anova.gexf".format(dir, group[0]))\n\ndf_edges_filter = nx.to_pandas_edgelist(H)\n\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-std-anova.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 2.68 ms (started: 2023-06-05 18:52:06 -05:00)


### ANOVA-STD

ANOVA

In [15]:
# ANOVA
""" df_nodes_anova = get_nodes_anova(SG, dir, group)
df_nodes_anova

# Filter by ANOVA (p > 0.001)
df_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]
df_nodes_anova_filter
nodes = list(df_nodes_anova_filter.index)
df_nodes_anova_filter """

' df_nodes_anova = get_nodes_anova(SG, dir, group)\ndf_nodes_anova\n\n# Filter by ANOVA (p > 0.001)\ndf_nodes_anova_filter = df_nodes_anova[df_nodes_anova["p-value"] > 0.001]\ndf_nodes_anova_filter\nnodes = list(df_nodes_anova_filter.index)\ndf_nodes_anova_filter '

time: 3.9 ms (started: 2023-06-05 18:52:07 -05:00)


In [16]:
""" x = df_nodes_anova["p-value"]
plt.hist(x, bins=100)
plt.axvline(x=0.001, color="red", lw=1)
l = len(df_nodes_anova) - len(df_nodes_anova_filter)
t = len(df_nodes_anova)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_nodes_anova["p-value"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.001, color="red", lw=1)\nl = len(df_nodes_anova) - len(df_nodes_anova_filter)\nt = len(df_nodes_anova)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 4 ms (started: 2023-06-05 18:52:07 -05:00)


In [17]:
# H = SG.subgraph(nodes)

time: 392 µs (started: 2023-06-05 18:52:07 -05:00)


In [18]:
""" df_edges_filter = nx.to_pandas_edgelist(H)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(H)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 3.33 ms (started: 2023-06-05 18:52:07 -05:00)


In [19]:
""" H = nx.from_pandas_edgelist(df_edges_filter, "source", "target")
graph_detail(H)

list_graphs.append(H.copy())
nx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova.gexf".format(dir, group[0])) """

' H = nx.from_pandas_edgelist(df_edges_filter, "source", "target")\ngraph_detail(H)\n\nlist_graphs.append(H.copy())\nnx.write_gexf(H, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova.gexf".format(dir, group[0])) '

time: 2.56 ms (started: 2023-06-05 18:52:07 -05:00)


STD

In [20]:
# STD
""" df_edges_std = get_edges_std(H, dir, group, subgroups, ddof=0)
df_edges_std

# Filter by std (std < 0.3)
df_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]
df_edges_std_filter """

' df_edges_std = get_edges_std(H, dir, group, subgroups, ddof=0)\ndf_edges_std\n\n# Filter by std (std < 0.3)\ndf_edges_std_filter = df_edges_std[df_edges_std["std"] < 0.3]\ndf_edges_std_filter '

time: 2.74 ms (started: 2023-06-05 18:52:07 -05:00)


In [21]:
""" x = df_edges_std["std"]
plt.hist(x, bins=100)
plt.axvline(x=0.3, color="red", lw=1)
l = len(df_edges_std) - len(df_edges_std_filter)
t = len(df_edges_std)
plt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))
plt.grid()
plt.show() """

' x = df_edges_std["std"]\nplt.hist(x, bins=100)\nplt.axvline(x=0.3, color="red", lw=1)\nl = len(df_edges_std) - len(df_edges_std_filter)\nt = len(df_edges_std)\nplt.title("Loss: {} of {} ({}%)".format(l, t, round(l*100/t)))\nplt.grid()\nplt.show() '

time: 3.31 ms (started: 2023-06-05 18:52:07 -05:00)


In [22]:
""" G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])
graph_detail(G)

list_graphs.append(G.copy())
nx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova-std.gexf".format(dir, group[0])) """

' G = nx.from_pandas_edgelist(df_edges_std_filter, "source", "target", edge_attr=["weight"])\ngraph_detail(G)\n\nlist_graphs.append(G.copy())\nnx.write_gexf(G, "{}/output_greedy/graphs_filter/greedy_{}_graph-filter-anova-std.gexf".format(dir, group[0])) '

time: 3.13 ms (started: 2023-06-05 18:52:07 -05:00)


In [23]:
""" df_edges_filter = nx.to_pandas_edgelist(G)
df_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova-std.csv".format(dir, group[0]), index=False)
df_edges_filter """

' df_edges_filter = nx.to_pandas_edgelist(G)\ndf_edges_filter.to_csv("{}/output_greedy/edges_filter/greedy_{}_edge-filter-anova-std.csv".format(dir, group[0]), index=False)\ndf_edges_filter '

time: 2.46 ms (started: 2023-06-05 18:52:07 -05:00)


### Results

In [24]:
""" for graph in list_graphs:
    graph_detail(graph) """

' for graph in list_graphs:\n    graph_detail(graph) '

time: 3.07 ms (started: 2023-06-05 18:52:07 -05:00)
